# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os
import openai

load_dotenv()  # Load .env
openai.api_key = os.getenv("OPENAI_API_KEY")

if openai.api_key is None:
    raise ValueError("OPENAI_API_KEY not found! Check your .env file.")


In [3]:
#Function to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [4]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {
    'table': ['customers', 'orders', 'products'],
    'definition': [
        'Contains customer information such as name, email, and address.',
        'Stores order details including order date, customer_id, and total amount.',
        'List of products with product name, description, and price.'
    ]
}
df = pd.DataFrame(data)
print(df)

       table                                         definition
0  customers  Contains customer information such as name, em...
1     orders  Stores order details including order date, cus...
2   products  List of products with product name, descriptio...


In [5]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [6]:
print(text_tables)

customers: Contains customer information such as name, email, and address.
orders: Stores order details including order date, customer_id, and total amount.
products: List of products with product name, description, and price.


In [7]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [8]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(
    tables=text_tables,
    question="Which customers have placed orders for products over $100?"
)

In [9]:
print(return_OAI(pqt1))

NameError: name 'OpenAI' is not defined

In [ ]:
pqt3 = prompt_question_tables.format(
    tables=text_tables,
    question="List all products that have never been ordered."
)

In [ ]:
print(return_OAI(pqt3))

```json
{
    "tables": ["products", "orders"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [ ]:
# -----------------------------
# 1. Install required packages (if needed)
# -----------------------------
#pip install openai pandas python-dotenv

# -----------------------------
# 2. Load API key from .env
# -----------------------------
from dotenv import load_dotenv
import os
import openai
import pandas as pd
from datetime import datetime

# Load environment variables from .env
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

if openai.api_key is None:
    raise ValueError("OPENAI_API_KEY not found. Make sure it's set in .env file.")

# -----------------------------
# 3. Sample tables and definitions
# -----------------------------
tables_info = {
    "customers": "Contains customer information such as name, email, and address",
    "orders": "Stores order details including order date, customer_id, and total amount",
    "products": "List of products with product name, description, and price",
    "employees": "Contains employee info including name, role, department",
    "shipments": "Contains shipment details with order_id, date_shipped, carrier"
}

user_questions = [
    "List all orders with customer names for last month",
    "Which products are out of stock?",
    "Show the total sales per employee",
    "Get the shipment dates for order #1234"
]

# -----------------------------
# 4. Function to query GPT
# -----------------------------
def query_gpt(prompt):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role":"user","content":prompt}],
            temperature=0
        )
        return response.choices[0].message['content'].strip()
    except Exception as e:
        return f"Error: {str(e)}"

# -----------------------------
# 5. Experiment: prompt strategies
# -----------------------------
results = []

for question in user_questions:
    # Strategy 1: tables only
    prompt1 = f"""You are a SQL assistant.
    User question: "{question}"
    Given the following tables: {list(tables_info.keys())}
    Select which tables are necessary to answer the question and explain briefly."""
    
    # Strategy 2: tables + descriptions
    prompt2 = f"""You are a SQL assistant.
    User question: "{question}"
    Here are tables with short descriptions: {tables_info}
    Select which tables are necessary to answer the question and explain briefly."""
    
    # Query GPT
    res1 = query_gpt(prompt1)
    res2 = query_gpt(prompt2)
    
    # Append results
    results.append({
        "question": question,
        "prompt_strategy": "tables_only",
        "model": "gpt-3.5-turbo",
        "selected_tables": res1,
        "comment": "" if "Error" not in res1 else "Failed"
    })
    
    results.append({
        "question": question,
        "prompt_strategy": "tables_with_descriptions",
        "model": "gpt-3.5-turbo",
        "selected_tables": res2,
        "comment": "" if "Error" not in res2 else "Failed"
    })

# -----------------------------
# 6. Convert results to DataFrame
# -----------------------------
df = pd.DataFrame(results)

# -----------------------------
# 7. Print results
# -----------------------------
print(df)

# -----------------------------
# 8. Save CSV report
# -----------------------------
folder_path = r"C:\Users\Lovely\Desktop\clean_code_experiment"
os.makedirs(folder_path, exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
file_path = os.path.join(folder_path, f"gpt_table_selection_report_{timestamp}.csv")
df.to_csv(file_path, index=False)
print(f"\nCSV report saved at: {file_path}")


                                            question  \
0  List all orders with customer names for last m...   
1  List all orders with customer names for last m...   
2                   Which products are out of stock?   
3                   Which products are out of stock?   
4                  Show the total sales per employee   
5                  Show the total sales per employee   
6             Get the shipment dates for order #1234   
7             Get the shipment dates for order #1234   

            prompt_strategy          model  \
0               tables_only  gpt-3.5-turbo   
1  tables_with_descriptions  gpt-3.5-turbo   
2               tables_only  gpt-3.5-turbo   
3  tables_with_descriptions  gpt-3.5-turbo   
4               tables_only  gpt-3.5-turbo   
5  tables_with_descriptions  gpt-3.5-turbo   
6               tables_only  gpt-3.5-turbo   
7  tables_with_descriptions  gpt-3.5-turbo   

                                     selected_tables comment  
0  Error: \n\nYou